In [1]:
import json
import pandas as pd
import numpy as np
from collections import Counter
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk import pos_tag
from string import punctuation,digits
import re
# from keras.utils import to_categorical
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import to_categorical
from keras.models import load_model
# from keras.layers import Input
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Flatten
# from keras.layers import Bidirectional
# from keras.layers import Embedding
# from keras.utils.np_utils import probas_to_classes
from sklearn import preprocessing
# from keras.models import Sequential
# from sklearn.metrics import confusion_matrix,f1_score,precision_score,recall_score,accuracy_score
# from sklearn.model_selection import train_test_split
# %matplotlib inline
# from keras.optimizers import Adam
# import matplotlib.pyplot as plt
# from matplotlib import rcParams
# from random import shuffle
# figure size in inches
# rcParams['figure.figsize'] = 11.7,8.27
import word2vecReader as godin_embedding
from gensim.models import KeyedVectors

In [2]:
# seed = 7
# np.random.seed(seed)

In [3]:
def load_data_from_json(filename):
    with open(filename,'r') as f:
        return json.load(f)

In [5]:
master_dict = load_data_from_json('dataset/master_test.json')

In [6]:
master_dict

{'0_Cuadrilla': {'info': [{'snippets': "['files to delay application']",
    'target': 'Cuadrilla'}],
  'sentence': 'Cuadrilla files to delay application to frack in Lancashire'},
 '1001_Sainsbury': {'info': [{'snippets': "['warns of squeeze on high street retailers']",
    'target': 'Sainsbury'}],
  'sentence': 'Sainsbury chief warns of squeeze on high street retailers'},
 '1006_Barclays': {'info': [{'snippets': "['fined for anti-money-laundering failings']",
    'target': 'Barclays'}],
  'sentence': 'Barclays fined for anti-money-laundering failings'},
 '1007_Barclays': {'info': [{'snippets': "['fined for lax crime checks in']",
    'target': 'Barclays'}],
  'sentence': "UPDATE 3-Barclays fined for lax crime checks in 'deal of century'"},
 '1014_GSK': {'info': [{'snippets': "['file up to 20 new drugs for approval by 2020']",
    'target': 'GSK'}],
  'sentence': 'GSK aims to file up to 20 new drugs for approval by 2020'},
 '1031_National Grid': {'info': [{'snippets': "['Grid lines up 

In [7]:
def extract_data_from_dict(master_dict):
    sentence_id = []
    snippet = []
    for key in master_dict.keys():
        for info in master_dict[key]['info']:
            snippet.append(eval(info['snippets'])[0])
            sentence_id.append(key)
    return snippet,sentence_id

In [8]:
snippet,sentence_id = extract_data_from_dict(master_dict)

In [9]:
len(snippet),len(sentence_id)

(192, 192)

In [10]:
def remove_punctuation(s):
    list_punctuation = list(punctuation.replace('$',''))
    for i in list_punctuation:
        s = s.replace(i,'')
    return s

In [13]:
def clean_sentence(sentence):
    #remove multiple repeat non num-aplha char !!!!!!!!!-->!
    sentence = re.sub(r'(\W)\1{2,}', r'\1', sentence) 
    #removes alpha char repeating more than twice aaaa->aa
    sentence = re.sub(r'(\w)\1{2,}', r'\1\1', sentence)
    #removes links
    sentence = re.sub(r'(?P<url>https?://[^\s]+)', r'', sentence)
    # remove @usernames
    sentence = re.sub(r"\@(\w+)", "", sentence)
    #remove # from #tags
    sentence = sentence.replace('#','')
    # split into tokens by white space
    tokens = sentence.split()
    # remove punctuation from each token
    tokens = [remove_punctuation(w) for w in tokens]
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    tokens = ' '.join(tokens)
    return tokens

In [15]:
dataX = [clean_sentence(x) for x in snippet]

In [16]:
max_length  = 11

In [17]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [19]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    encoded = tokenizer.texts_to_sequences(lines)
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [20]:
tokenizer = create_tokenizer(dataX)
vocab_size = len(tokenizer.word_index) + 1
trainX = encode_text(tokenizer, dataX, max_length)

In [21]:
model = load_model('models/aspect_model.h5')

In [23]:
pred = model.predict(trainX)

In [27]:
with open('models/l2_l1_map.json','r') as fin:
    L2_L1_pair = json.load(fin)

In [29]:
with open('models/label_encoding.json','r') as fin:
    enc = json.load(fin)

In [31]:
lable_encoding = enc['lables_index']

In [33]:
def get_class_from_pred(pred):
    return [lable_encoding[x.argmax()] for x in pred]

In [34]:
pred_class = get_class_from_pred(pred)

In [35]:
pred_class

['Financial',
 'Price Action',
 'Price Action',
 'Strategy',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Dividend Policy',
 'Technical Analysis',
 'Coverage',
 'Price Action',
 'Coverage',
 'Technical Analysis',
 'Price Action',
 'Strategy',
 'Price Action',
 'Price Action',
 'Price Action',
 'IPO',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Sales',
 'Price Action',
 'Price Action',
 'M&A',
 'Price Action',
 'Dividend Policy',
 'Appointment',
 'Technical Analysis',
 'Coverage',
 'IPO',
 'Signal',
 'Appointment',
 'Technical Analysis',
 'Sales',
 'Market',
 'M&A',
 'Dividend Policy',
 'Sales',
 'Price Action',
 'Price Action',
 'Company Communication',
 'Sales',
 'Risks',
 'Sales',
 'Price Action',
 'Dividend Policy',
 'M&A',
 'Rumors',
 'Price Action',
 'Technical Analysis',
 'M&A',
 'Price Action',
 'Price Action',
 'Price Action',
 'Rumors',
 'Price Action',
 'Sales',
 'Coverage',
 'Price Action',
 'Sales',
 'Options',
 'Volatility',
 'Price Action'

In [43]:
def combine_level2_and_level1(pred_class):
    aspect = []
    for level2 in pred_class:
        aspect.append(L2_L1_pair[level2][0]+"/"+level2)
    return aspect

In [44]:
aspect = combine_level2_and_level1(pred_class)

In [46]:
len(aspect),len(sentence_id),len(snippet)

(192, 192, 192)

In [60]:
result = {'results':[]}

In [61]:
for s_id,s_sn,asp in zip(sentence_id,snippet,aspect):
    result['results'].append({'id':s_id,'snippet':s_sn,'aspect_categories':asp})

In [62]:
result

{'results': [{'aspect_categories': 'Corporate/Financial',
   'id': '0_Cuadrilla',
   'snippet': 'files to delay application'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1001_Sainsbury',
   'snippet': 'warns of squeeze on high street retailers'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1006_Barclays',
   'snippet': 'fined for anti-money-laundering failings'},
  {'aspect_categories': 'Corporate/Strategy',
   'id': '1007_Barclays',
   'snippet': 'fined for lax crime checks in'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1014_GSK',
   'snippet': 'file up to 20 new drugs for approval by 2020'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1031_National Grid',
   'snippet': 'Grid lines up sale of'},
  {'aspect_categories': 'Corporate/M&A',
   'id': '1034_British American Tobacco',
   'snippet': 'accused of bribing senior politicians to sabotage'},
  {'aspect_categories': 'Stock/Price Action',
   'id': '1035_MillerCoors',
   'snippet': 'l

In [63]:
with open('results/aspect_result.json','w') as fout:
    json.dump(result,fout,indent=4)

In [92]:
para_em = Categorical(categories=['embedding_matrix_fast_text','embedding_matrix_godin','embedding_matrix_word2vec','embedding_matrix_glove','free'],name='em')

In [93]:
para_em_trainable_flag = Categorical(categories=[True,False],name='em_trainable_flag')

In [94]:
para_free_em_dim = Categorical(categories=[100,300,400],name='free_em_dim')

In [99]:
para_batch_size = Categorical(categories=[8,16,32,64],name='batch_size')

In [96]:
para_epoch = Categorical(categories=[10,50,70,100,150],name='epoch')

In [97]:
parameters = [para_learning_rate,para_dropout,para_lstm_out,para_n_hidden_layer,para_em,para_em_trainable_flag,para_free_em_dim,para_batch_size,para_epoch]

In [98]:
default_parameters = [0.0006,0.25,600,1,'embedding_matrix_word2vec',True,100,64,70]

In [101]:
def define_model(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,free_em_dim):
    model = Sequential()
    
    if em == 'free':
        model.add(Embedding(vocab_size, free_em_dim))
    else:
        model.add(Embedding(vocab_size, len(eval(em)[0]), weights = [eval(em)],input_length=max_length,trainable = em_trainable_flag))
    model.add(Bidirectional(LSTM(lstm_out, recurrent_dropout=dropout)))
    for i in range(n_hidden_layer):
        model.add(Dense(int((2*lstm_out+no_of_classes)/2),activation='relu'))
    model.add(Dense(no_of_classes,activation='softmax'))
    optimizer = Adam(lr=learning_rate)
    model.compile(loss = 'categorical_crossentropy', optimizer=optimizer,metrics = ['accuracy'])
#     print(model.summary())
    return model

In [102]:
#dict to store performance of all models
record = dict()
key=0

In [81]:
def get_class_from_pred(pred):
    return [lable_encoding[x.argmax()] for x in pred]

In [103]:
# def print_metric(yacc,ypred,avg_type=None):
#     print('---------------------avg type = {0}---------------------------------------------'.format(avg_type))
#     print('F1 score == ', f1_score(yacc,ypred,average=avg_type))
#     print('precision score == ', precision_score(yacc,ypred,average=avg_type))
#     print('recall score == ', recall_score(yacc,ypred,average=avg_type))
#     print('accuracy score == ', accuracy_score(yacc,ypred))

In [83]:
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)

In [84]:
dataX = np.asarray(dataX)
dataY = np.asarray(dataY)

In [ ]:
@use_named_args(dimensions=parameters)
def fitness(learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,free_em_dim,batch_size,epoch):
# n_dense,dropout,learning_rate,n_filters,filter_size_c1,filter_size_c2,filter_size_c3,em_c1,em_c2,em_c3,free_em_dim,em_trainable_flag
    # Print the hyper-parameters.
    global key
    global record
    print('-----------------------------combination no={0}------------------'.format(key))
    print('learning rate ==>',learning_rate)
    print('dropout==>',dropout)
    print('lstm_out==>',lstm_out)
    print('n_hidden_layer==>',n_hidden_layer)
    print('em==>',em)
    print('em_trainable_flag ==>',em_trainable_flag)
    print('free_em_dim==>',free_em_dim)
    print('batch_size==>',batch_size)
    print('epocs==>',epoch)
    
    cv_f1 = []
    first_iteration_flag = True
    for train,test in kfold.split(dataX,dataY):
        trainX = list(dataX[train])
        trainY = list(dataY[train])
        testX = list(dataX[test])
        testY = list(dataY[test])
        tokenizer = create_tokenizer(trainX)
        vocab_size = len(tokenizer.word_index) + 1
        trainX = encode_text(tokenizer, trainX, max_length)
        testX = encode_text(tokenizer, testX, max_length)
        trainY,lable_encoding = convert_lables(trainY,testY)
        if first_iteration_flag:
            with open('models/label_encoding.json','w') as f:
                json.dump({'lables_index':list(lable_encoding)},f)
            first_iteration_flag = False
        embedding_matrix_glove = get_GloVe_embedding_matrix(embeddings_index_glove)
        embedding_matrix_word2vec = get_word2vec_embedding_matrix(word2vec_model)
        embedding_matrix_fast_text = get_fast_text_matrix(fast_text_model)
        embedding_matrix_godin = get_godin_embedding_matrix(godin_model)
        model = define_model(learning_rate=learning_rate,
                         dropout=dropout,
                         lstm_out=lstm_out,
                         n_hidden_layer=n_hidden_layer,
                         em=em,
                         em_trainable_flag=em_trainable_flag,
                         free_em_dim=free_em_dim)
        
        history_object = model.fit(trainX, trainY, epochs=epoch,batch_size=batch_size)
        pred = model.predict(testX)
        pred_class = get_class_from_pred(pred)
        f1 = f1_score(testY,pred_class,average='weighted')
        print(f1)
        cv_f1.append(f1)
        
    f1_mean = np.mean(cv_f1)
    print(cv_f1)
    print(f1_mean)
    
    record[key] = {'parameters':[learning_rate,dropout,lstm_out,n_hidden_layer,em,em_trainable_flag,free_em_dim,batch_size,epoch],'F1':f1_mean,'cv_f1':cv_f1}
    with open('models/record.json', 'w') as fout:
        json.dump(record,fout,indent=4)
    
    
    key+=1
    
    # Delete the Keras model with these hyper-parameters from memory.
    del model
    
    # Clear the Keras session, otherwise it will keep adding new
    # models to the same TensorFlow graph each time we create
    # a model with a different set of hyper-parameters.
    K.clear_session()
    
    # NOTE: Scikit-optimize does minimization so it tries to
    # find a set of hyper-parameters with the LOWEST fitness-value.
    # Because we are interested in the HIGHEST classification
    # accuracy, we need to negate this number so it can be minimized.
    return -f1_mean

In [ ]:
search_result = gp_minimize(func=fitness,
                            dimensions=parameters,
                            acq_func='EI',
                            n_calls=11,
                            x0=default_parameters)

In [86]:
# first_iteration_flag = True
# for train,test in kfold.split(dataX,dataY):
#     trainX = list(dataX[train])
#     trainY = list(dataY[train])
#     testX = list(dataX[test])
#     testY = list(dataY[test])
#     tokenizer = create_tokenizer(trainX)
#     vocab_size = len(tokenizer.word_index) + 1
#     trainX = encode_text(tokenizer, trainX, max_length)
#     testX = encode_text(tokenizer, testX, max_length)
#     trainY,lable_encoding = convert_lables(trainY,testY)
#     if first_iteration_flag:
#         with open('models/label_encoding.json','w') as f:
#             json.dump({'lables_index':list(lable_encoding)},f)
#         first_iteration_flag = False
#     embedding_matrix_word2vec = get_word2vec_embedding_matrix(word2vec_model)
#     model = define_model(learning_rate=0.0006,
#                          dropout=0.25,
#                          lstm_out=600,
#                          n_hidden_layer=1,
#                          em='embedding_matrix_word2vec',
#                          em_trainable_flag=True,
#                          free_em_dim=100)
#     history_object = model.fit(trainX, trainY, epochs=70,batch_size=64)
#     pred = model.predict(testX)
#     pred_class = get_class_from_pred(pred)
#     f1 = f1_score(testY,pred_class,average='weighted')
#     print(f1)
#     cv_f1.append(f1)

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Epoch 1/70
1049/1049 [==============================] - 14s 14ms/step - loss: 2.6727 - acc: 0.3622
Epoch 2/70
1049/1049 [==============================] - 13s 13ms/step - loss: 1.9034 - acc: 0.5005
Epoch 3/70
1049/1049 [==============================] - 13s 13ms/step - loss: 1.3788 - acc: 0.6168
Epoch 4/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.9689 - acc: 0.7350
Epoch 5/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.6578 - acc: 0.8303
Epoch 6/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.5034 - acc: 0.8618
Epoch 7/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.3370 - acc: 0.9123
Epoch 8/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.2191 - acc: 0.9447
Epoch 9/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.1757 - acc: 0.9542
Epoch 10/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.1243 - acc: 0.9676
Epoch 11/

/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/hitkul/anaconda3/envs/ps3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


0.6407337856051247
Epoch 1/70
1049/1049 [==============================] - 14s 14ms/step - loss: 2.7237 - acc: 0.3527
Epoch 2/70
1049/1049 [==============================] - 13s 13ms/step - loss: 1.9791 - acc: 0.4795
Epoch 3/70
1049/1049 [==============================] - 13s 13ms/step - loss: 1.4651 - acc: 0.5968
Epoch 4/70
1049/1049 [==============================] - 13s 13ms/step - loss: 1.0375 - acc: 0.7054
Epoch 5/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.7518 - acc: 0.7922
Epoch 6/70
1049/1049 [==============================] - 14s 13ms/step - loss: 0.5504 - acc: 0.8313
Epoch 7/70
1049/1049 [==============================] - 14s 13ms/step - loss: 0.3419 - acc: 0.8990
Epoch 8/70
1049/1049 [==============================] - 14s 13ms/step - loss: 0.3329 - acc: 0.9085
Epoch 9/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.2377 - acc: 0.9361
Epoch 10/70
1049/1049 [==============================] - 13s 13ms/step - loss: 0.1770 - ac

KeyboardInterrupt: 

In [65]:
# model = define_model(learning_rate=0.0006,
#                          dropout=0.25,
#                          lstm_out=600,
#                          n_hidden_layer=1,
#                          em='embedding_matrix_word2vec',
#                          em_trainable_flag=True,
#                          free_em_dim=100)

In [66]:
# history_object = model.fit(trainX, trainY, epochs=70,batch_size=64)

In [67]:
# model.save('models/bestAspect_snippt_master.h5')

In [68]:
# pred = model.predict(testX)

In [142]:
# pred_class = get_class_from_pred(pred)

In [143]:
#get metric scores


In [70]:
# print_metric(testY,pred_class)
# print_metric(testY,pred_class,'micro')
# print_metric(testY,pred_class,'macro')
# print_metric(testY,pred_class,'weighted')